In [1]:
import pandas as pd
import pyreadstat
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 

import econtools as econ
import econtools.metrics as mt
import math


In [2]:
# 3차 
df_HH3, meta = pyreadstat.read_sav("C:/Users/thxkn/OneDrive/바탕 화면/CV/박사지원서류/WS_HJ/KEEP/KEEPData_1-12Wave_Ver12/DATA(수능, 직업력, 가중치, 패널조사현황 자료 포함)/1-12차년도 DATA(SPSS)/3차년도(2006) DATA/중학교 3학년생 코호트/f2_m_household.sav")

#소득 [F2H04001]
df_HH3["F2H04001"].describe()
df_HH3 = df_HH3.rename(columns = {"F2H04001": "income"})
df_HH3 = df_HH3[df_HH3["income"]>0]
df_HH3["year"] = 2003


In [3]:
# 4차
df_HH4, meta = pyreadstat.read_sav("C:/Users/thxkn/OneDrive/바탕 화면/CV/박사지원서류/WS_HJ/KEEP/KEEPData_1-12Wave_Ver12/DATA(수능, 직업력, 가중치, 패널조사현황 자료 포함)/1-12차년도 DATA(SPSS)/4차년도(2007) DATA/중학교 3학년생 코호트/f3_m_household.sav")

#소득 F3H08001
df_HH4["F3H08001"].describe()
df_HH4 = df_HH4.rename(columns = {"F3H08001": "income"})
df_HH4 = df_HH4[df_HH4["income"]>0]
df_HH4["year"] = 2004

In [4]:
df_HH = pd.concat([df_HH3, df_HH4])
len(df_HH)

4057

In [100]:
# 3차_학생 고등학교 일반계 , 전문계

df_st3, meta = pyreadstat.read_sav("C:/Users/thxkn/OneDrive/바탕 화면/CV/박사지원서류/WS_HJ/KEEP/KEEPData_1-12Wave_Ver12/DATA(수능, 직업력, 가중치, 패널조사현황 자료 포함)/1-12차년도 DATA(SPSS)/3차년도(2006) DATA/중학교 3학년생 코호트/f2_m_highschool_st.sav")
df_st_v3, meta = pyreadstat.read_sav("C:/Users/thxkn/OneDrive/바탕 화면/CV/박사지원서류/WS_HJ/KEEP/KEEPData_1-12Wave_Ver12/DATA(수능, 직업력, 가중치, 패널조사현황 자료 포함)/1-12차년도 DATA(SPSS)/3차년도(2006) DATA/중학교 3학년생 코호트/f2_m_vocationalhighschool_st.sav")
df3 = pd.concat([df_st3, df_st_v3])
df3["year"] = 2003

In [101]:
df3 = df3.rename(columns={'F2S01003':'typeofschool'})
df3 = df3.rename(columns={'F2S01002':'region'})

In [102]:
# 4차_학생 고등학교 일반계, 전문계

df_st4, meta = pyreadstat.read_sav("C:/Users/thxkn/OneDrive/바탕 화면/CV/박사지원서류/WS_HJ/KEEP/KEEPData_1-12Wave_Ver12/DATA(수능, 직업력, 가중치, 패널조사현황 자료 포함)/1-12차년도 DATA(SPSS)/4차년도(2007) DATA/중학교 3학년생 코호트/f3_m_highschool_st.sav")
df_st_v4, meta = pyreadstat.read_sav("C:/Users/thxkn/OneDrive/바탕 화면/CV/박사지원서류/WS_HJ/KEEP/KEEPData_1-12Wave_Ver12/DATA(수능, 직업력, 가중치, 패널조사현황 자료 포함)/1-12차년도 DATA(SPSS)/4차년도(2007) DATA/중학교 3학년생 코호트/f3_m_vocationalhighschool_st.sav")
df4 = pd.concat([df_st4, df_st_v4])
df4["year"] = 2004
df4 = df4.rename(columns={'F3S01003':'typeofschool'})
df4 = df4.rename(columns={'F3S01002':'region'})

In [137]:
df = pd.concat([df3, df4])

In [138]:
len(df)

4715

In [139]:
df['year'].value_counts()

2004    2976
2003    1739
Name: year, dtype: int64

In [140]:
# 일반계 (1), 전문계 (0) 표시
hs_list3 = df_st3["BYSID"].to_list()
hs_list4 = df_st4["BYSID"].to_list()
df["hs_group"] = 0
df.loc[df["BYSID"].isin(hs_list3) == True, "hs_group"] = 1
df.loc[df["BYSID"].isin(hs_list4) == True, "hs_group"] = 1
df["hs_group"].value_counts()

1    2857
0    1858
Name: hs_group, dtype: int64

<h2>Main Dataset</h2>

In [141]:
df = pd.merge(df, df_HH)
#df3 does not have information about teacehrs #mighbe use 1차년도 teacher (2차년도-고1-teacher도 없음)

In [142]:
df

,BYSID,BYHID,GENDER,BYSCLASS,region,typeofschool,F2S01004,F2S01005,F2S01006,F2S01007,...,F3H12022,F3H12023,F3H12024,F3H12025,F3H12026,F3H12027,F3H13001,F3H13002,F3H13003,F3H13004
0,1003.0,10031.0,1.0,1.0,8.0,2.0,1.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1005.0,10051.0,1.0,1.0,15.0,2.0,1.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1007.0,10071.0,1.0,1.0,2.0,2.0,2.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1008.0,10081.0,1.0,1.0,8.0,2.0,2.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1010.0,10101.0,1.0,1.0,8.0,2.0,1.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3854,7947.0,79471.0,2.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,...,-5.0,-5.0,-5.0,-5.0,1.0,-2.0,3.0,1.0,2.0,1.0
3855,7948.0,79481.0,1.0,1.0,8.0,NaN,NaN,NaN,NaN,NaN,...,-5.0,-5.0,-5.0,-5.0,1.0,2.0,1.0,1.0,0.0,1.0
3856,7949.0,79491.0,2.0,1.0,15.0,NaN,NaN,NaN,NaN,NaN,...,-5.0,-5.0,-5.0,-5.0,2.0,2.0,2.0,0.0,2.0,1.0
3857,7950.0,79501.0,2.0,1.0,4.0,NaN,NaN,NaN,NaN,NaN,...,-5.0,-5.0,-5.0,-5.0,1.0,2.0,2.0,0.0,2.0,2.0


In [143]:
# 남 1, 여 0
df.loc[df['GENDER']==2.0, 'GENDER'] =0

In [144]:
# income rank 03 
a = df.loc[df['year'] == 2003, "income"]

df["income_rank_p"] = np.nan
df.loc[ df['year'] == 2003 , "income_rank_p"] = a.rank(method = 'dense') 
df.loc[ df['year'] == 2003 , "income_rank_p"] = df.loc[ df['year'] == 2003 , "income_rank_p"]/ df.loc[ df['year'] == 2003 , "income_rank_p"].max()

In [145]:
# income rank 04
a = df.loc[df['year'] == 2004, "income"]

df.loc[ df['year'] == 2004 , "income_rank_p"] = a.rank(method = 'dense') 
df.loc[ df['year'] == 2004 , "income_rank_p"] = df.loc[ df['year'] == 2004 , "income_rank_p"]/ df.loc[ df['year'] == 2004 , "income_rank_p"].max()

In [146]:
df["income_rank_p"].isna().value_counts()

False    3859
Name: income_rank_p, dtype: int64

<h2>Achievement Goal</h2>

In [147]:
# achievement goal
# 3차
df = df.rename(columns = {'F2S01071': 'learning', 'F2S01072': 'betterperson', 'F2S01073': 'betterjob', 'F2S01074': 'futurework', 'F2S01075': 'forsociety', 'F2S01076': 'favorablejudg', 'F2S01077': 'popular', 'F2S01078': 'competition', 'F2S01079': 'forced', 'F2S01080': 'negativejudg', 'F2S01081': 'penalty'})

In [148]:
# 4차 90은 money인데 3차에는 없음
df.loc[df['year'] == 2004, 'learning'] = df['F3S01087']
df.loc[df['year'] == 2004, 'betterperson'] = df['F3S01088']
df.loc[df['year'] == 2004, 'betterjob'] = df['F3S01089']
df.loc[df['year'] == 2004, 'futurework'] = df['F3S01091']
df.loc[df['year'] == 2004, 'forsociety'] = df['F3S01092']
df.loc[df['year'] == 2004, 'favorablejudg'] = df['F3S01093']
df.loc[df['year'] == 2004, 'popular'] = df['F3S01094']
df.loc[df['year'] == 2004, 'competition'] = df['F3S01095']
df.loc[df['year'] == 2004, 'forced'] = df['F3S01096']
df.loc[df['year'] == 2004, 'negativejudg'] = df['F3S01097']
df.loc[df['year'] == 2004, 'penalty'] = df['F3S01098']

In [149]:
# achievement goal
# 3차
goal_list = ['learning', 'betterperson', 'betterjob', 'futurework', 'forsociety', 'favorablejudg',  'popular',  'competition', 'forced', 'negativejudg',  'penalty']
for i in goal_list:
    df = df[df[i]>0]

In [150]:
drop_list = []

for i in range(87, 99):
    df = df.drop('F3S010'+str(i), axis = 1)

In [159]:
len(df)

3848

In [152]:
df.loc[df['hs_group'] == 0, 'typeofschool'] = 0
df = df[df['typeofschool']>=0]

In [163]:
pyreadstat.write_sav(df, 'df.sav')

In [164]:
id_list = df3['BYSID'].to_list()

In [166]:
df.loc[df['BYSID'] == 1003.0]

,BYSID,BYHID,GENDER,BYSCLASS,region,typeofschool,F2S01004,F2S01005,F2S01006,F2S01007,...,F3H12023,F3H12024,F3H12025,F3H12026,F3H12027,F3H13001,F3H13002,F3H13003,F3H13004,income_rank_p
0,1003.0,10031.0,1.0,1.0,8.0,2.0,1.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.547619
1521,1003.0,10031.0,1.0,1.0,8.0,7.0,NaN,NaN,NaN,NaN,...,-5.0,-5.0,-5.0,1.0,2.0,2.0,1.0,1.0,2.0,0.369748
